In [1]:
import pandas as pd
import numpy as np
import math
from scipy.stats.mstats import winsorize
from openpyxl import load_workbook



In [3]:
# cargar  archivos

df_items = pd.read_excel(r"C:/Users/JORGE CONTRERAS/OneDrive - 900208659-2 DAMIS SAS/Escritorio/PLANEACION/PLANEACION/INDICADORES PLANEACION/analisis_pedidos.xlsx",sheet_name='ITEMS')
df_consumos = pd.read_excel(r"C:/Users/JORGE CONTRERAS/OneDrive - 900208659-2 DAMIS SAS/Escritorio/PLANEACION/PLANEACION/INDICADORES PLANEACION/analisis_pedidos.xlsx",sheet_name='CONSUMOS',index_col='Año')
df_lista_materiales = pd.read_excel(r"C:/Users/JORGE CONTRERAS/OneDrive - 900208659-2 DAMIS SAS/Escritorio/PLANEACION/PLANEACION/INDICADORES PLANEACION/analisis_pedidos.xlsx",sheet_name='lista_materiales')
df_inventarios = pd.read_excel(r"C:/Users/JORGE CONTRERAS/OneDrive - 900208659-2 DAMIS SAS/Escritorio/PLANEACION/PLANEACION/INDICADORES PLANEACION/analisis_pedidos.xlsx",sheet_name='INVENTARIOS')
df_pedidos = pd.read_excel(r"C:/Users/JORGE CONTRERAS/OneDrive - 900208659-2 DAMIS SAS/Escritorio/PLANEACION/PLANEACION/INDICADORES PLANEACION/analisis_pedidos.xlsx",sheet_name='Pedidos_clientes')
df_solicitudes = pd.read_excel(r"C:/Users/JORGE CONTRERAS/OneDrive - 900208659-2 DAMIS SAS/Escritorio/PLANEACION/PLANEACION/INDICADORES PLANEACION/analisis_pedidos.xlsx",sheet_name='solicitudes')
df_OCC = pd.read_excel(r"C:/Users/JORGE CONTRERAS/OneDrive - 900208659-2 DAMIS SAS/Escritorio/PLANEACION/PLANEACION/INDICADORES PLANEACION/analisis_pedidos.xlsx",sheet_name='ordenes_compra')
df_OCC_HISTORICO = pd.read_excel(r"C:/Users/JORGE CONTRERAS/OneDrive - 900208659-2 DAMIS SAS/Escritorio/PLANEACION/PLANEACION/INDICADORES PLANEACION/analisis_pedidos.xlsx",sheet_name='HISTORICO OCC')

print(df_consumos.columns)


Index(['Mes', 'Referencia', 'Fecha', 'Documento', 'Desc. item', 'Bodega',
       'Lote', 'Ubicación', 'Entradas (inv.)', 'Salidas (inv.)'],
      dtype='object')


TABLA FRECUENCIA DE CONSUMOS MES A MES

In [7]:
# Asegúrate de que 'Fecha' esté en formato datetime
df_consumos['Fecha'] = pd.to_datetime(df_consumos['Fecha'], errors='coerce')
df_consumos['Mes-Año'] = df_consumos['Fecha'].dt.strftime('%b-%y')

# Filtra solo las fechas de los años 2024 y 2025
filtro_año_2024 = df_consumos['Fecha'].dt.year == 2024
filtro_año_2025 = df_consumos['Fecha'].dt.year == 2025

# Filtra los datos para 2024 y 2025 por separado
df_consumos_2024 = df_consumos[filtro_año_2024]
df_consumos_2025 = df_consumos[filtro_año_2025]

# Función para agrupar y pivotar datos
def procesar_datos(df):
    df = df.groupby(['Mes-Año', 'Referencia', 'Desc. item', 'Bodega'])['Salidas (inv.)'].sum().reset_index()
    df = df.pivot_table(index=['Referencia', 'Desc. item', 'Bodega'], columns='Mes-Año', values='Salidas (inv.)', aggfunc='sum').reset_index()
    df = df.fillna(0)
    
    columnas_consumo = df.columns[3:]
    df['Frecuencia_consumo'] = (df[columnas_consumo] > 0).sum(axis=1)
    
    return df

# Procesar los datos de 2024 y 2025
df_consumos_2024_procesado = procesar_datos(df_consumos_2024)
df_consumos_2025_procesado = procesar_datos(df_consumos_2025)

# Mostrar los resultados
print("Tabla para 2024:")
print(df_consumos_2024_procesado)

print("Tabla para 2025:")
print(df_consumos_2025_procesado)

# Guardar los resultados en archivos Excel separados
df_consumos_2024_procesado.to_excel('frecuencia_2024.xlsx', index=False)
df_consumos_2025_procesado.to_excel('frecuencia_2025.xlsx', index=False)





Tabla para 2024:
Mes-Año Referencia                                Desc. item Bodega  Apr-24  \
0                3                  CARTON SINGLE FACE KILOS  MP001  354.00   
1                3                  CARTON SINGLE FACE KILOS  MP002  839.00   
2                8   CINTA EMPAQUE IMPRESA 48 mm *100 MTS C7  MP001   46.00   
3                8   CINTA EMPAQUE IMPRESA 48 mm *100 MTS C7  MP002  167.00   
4                9   CINTA EMPAQUE  2 * 100 M TOLDOS TOSCANI  MP002   30.00   
...            ...                                       ...    ...     ...   
1950       TUBSM03     TUBO RANURADO 70mm 4m (66015) TOSCANY  MP002   10.50   
1951       TUBSM04  TUBO CUADRADO 40X40mm 5.85m (66016)TOSCA  MP002   15.75   
1952       TUBSM05   TUBO CUADRADO 40X40mm 5m (66016)TOSCANY  MP002   13.50   
1953       TUBSM06   TUBO CUADRADO 40X40mm 4m (66016)TOSCANY  MP002    0.00   
1954       TUBSM07   TUBO RANURADO 80mm 5.8 Mts (66015)TOSCN  MP002   29.25   

Mes-Año  Aug-24  Dec-24  Feb-24  J

TABLA CONSUMOS MAXIMOS-MINIMOS


In [17]:
df_total_consumos = df_consumos.copy()
#df_total_consumos_mes = df_consumos.copy()
#df_total_consumos.to_excel('total_consumos_salida_pronT.xlsx', index=False)
df_total_consumos_normalizado = df_consumos.copy()




def winsorize_group(serie):
    return winsorize(serie, limits=[0.05, 0.05])  # Ajusta el umbral si es necesario

# Aplicar por cada ITEM
df_total_consumos_normalizado['Salida_pron'] = (
    df_total_consumos.groupby(['Referencia','Bodega'])['Salidas (inv.)']
    .transform(lambda x: winsorize_group(x))
)



print(df_total_consumos_normalizado.head(5))



df_total_consumos = df_total_consumos_normalizado.copy().groupby(['Año','Referencia','Desc. item','Bodega']).agg(
    Total_consumo=('Salida_pron','sum'),
    Consumo_min=('Salida_pron','min'),
    Consumo_max=('Salida_pron','max'),
    Consumo_promedio=('Salida_pron','mean'),
    desviacion_estandar=('Salida_pron','std')
).reset_index()


df_total_consumos.to_excel('total_consumos.xlsx', index=False)



df_total_consumos_mes = df_total_consumos_normalizado.copy().groupby(['Año','Mes-Año','Referencia','Desc. item','Bodega']).agg(
    Total_consumo=('Salidas (inv.)','sum'),
    Consumo_maximo=('Salidas (inv.)','max'),
    Consumo_minimo=('Salidas (inv.)','min'),
    Consumo_promedio=('Salidas (inv.)','mean'),
    desviacion_estandar=('Salidas (inv.)','std')
).reset_index()

df_total_consumos_mes['Consumo_promedio_mes'] = df_total_consumos_mes['Total_consumo']/25

df_total_consumos_mes = df_total_consumos_mes.groupby(['Año','Referencia','Desc. item','Bodega']).agg(
    Total_consumo=('Total_consumo','sum'),
    Consumo_maximo=('Consumo_maximo','max'),
    Consumo_minimo=('Consumo_minimo','min'),
    Consumo_promedio=('Consumo_promedio_mes','mean'),
    desviacion_estandar=('desviacion_estandar','std'),
    Consumo_promedio_mes_std=('Consumo_promedio_mes','std')
).reset_index()

print(df_total_consumos_mes.columns)
#df_total_consumos_mes =df_total_consumos_mes['Año'].dt.year.isin([2024, 2025])

df_total_consumos_mes.to_excel('total_consumos_mes.xlsx', index=False)


df_consumos_año= df_total_consumos.copy() #BD CON DESVIACION ESTANDAR Y CONSUMO PROMEDIO POR AÑO

df_consumos_año.to_excel('total_consumos_año.xlsx', index=False)



'''
df_total_consumos=df_total_consumos.loc[filtro_año]

df_total_consumos  #ESTA TABLA MANEJA EL TOTAL DE CONSUMOS DE CADA ITEM POR BODEGA

df_total_consumos1 = df_total_consumos_salida_pron.groupby(['Referencia', 'Desc. item', 'Bodega'])['Salida_pron'].sum().reset_index()

df_total_consumos = df_total_consumos.merge(df_total_consumos1, how='left', on=['Referencia', 'Desc. item', 'Bodega'])



print(df_total_consumos.columns)


df_total_consumos=df_total_consumos.groupby(['Referencia','Desc. item','Bodega']).agg({
    'Total_consumo':'mean',
    'Salida_pron':['min','max','mean','std','median']    
    }
)

df_total_consumos.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in df_total_consumos.columns]

print(df_total_consumos)
df_total_consumos

df_total_consumos.to_excel('val.xlsx')

'''













      Mes Referencia      Fecha     Documento  \
Año                                             
2022    1      18236 2022-01-03  CPM-00026873   
2022    1      18393 2022-01-03  CP -00028519   
2022    1      18474 2022-01-03  CP -00028519   
2022    1      18494 2022-01-03  CP -00028519   
2022    1        424 2022-01-03  SCI-00047772   

                                 Desc. item Bodega  Lote  Ubicación  \
Año                                                                   
2022  CASQUILLO ALUM 80mm L/OPUESTO TOSCANY  MP002   NaN        NaN   
2022    RECIBIDOR EN AC P/SOMRBILLA ALSACIA  MP001   NaN        NaN   
2022  TORN BRISTOL C/BOTON 5/16X1-1/2" INOX  MP001   NaN        NaN   
2022              CADENA NIQUELADA REF: 618  MP001   NaN        NaN   
2022            GUANTES VAQUETA MANGA CORTA  MP002   NaN        NaN   

      Entradas (inv.)  Salidas (inv.) Mes-Año  Salida_pron  
Año                                                         
2022                0             1.

"\ndf_total_consumos=df_total_consumos.loc[filtro_año]\n\ndf_total_consumos  #ESTA TABLA MANEJA EL TOTAL DE CONSUMOS DE CADA ITEM POR BODEGA\n\ndf_total_consumos1 = df_total_consumos_salida_pron.groupby(['Referencia', 'Desc. item', 'Bodega'])['Salida_pron'].sum().reset_index()\n\ndf_total_consumos = df_total_consumos.merge(df_total_consumos1, how='left', on=['Referencia', 'Desc. item', 'Bodega'])\n\n\n\nprint(df_total_consumos.columns)\n\n\ndf_total_consumos=df_total_consumos.groupby(['Referencia','Desc. item','Bodega']).agg({\n    'Total_consumo':'mean',\n    'Salida_pron':['min','max','mean','std','median']    \n    }\n)\n\ndf_total_consumos.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in df_total_consumos.columns]\n\nprint(df_total_consumos)\ndf_total_consumos\n\ndf_total_consumos.to_excel('val.xlsx')\n\n"

In [29]:
df_items_consolidado = df_items.copy()

print(df_items_consolidado.head(5))

df_total_consumos = df_total_consumos_mes[df_total_consumos_mes['Año'].isin([2024,2025])]

df_items_consolidado= df_items_consolidado.merge(df_total_consumos,how='left', on = ['Referencia','Desc. item','Bodega'])
df_items_consolidado.to_excel('items_consolidado.xlsx', index=False)

df_items_consolidado= df_items_consolidado.merge(df_inventarios[['Referencia','Desc. item','Bodega','Cant. disponible']],how='left', on=['Referencia','Desc. item','Bodega'])

df_items_consolidado['Cant. disponible'] = df_items_consolidado.apply(
    lambda row: row['Cant. disponible'] if row['Año'] == 2025 else 0, axis=1)

df_items_consolidado = df_items_consolidado.fillna(0)

df_items_consolidado1 =df_OCC.groupby(['Referencia','Desc. item','Bodega'])['Cant. pendiente'].sum().reset_index()
df_items_consolidado =df_items_consolidado.merge(df_items_consolidado1,how='left',on=['Referencia','Desc. item','Bodega'])

df_items_consolidado['Cant. pendiente'] = df_items_consolidado.apply(
    lambda row: row['Cant. pendiente'] if row['Año'] == 2025 else 0, axis=1)

df_items_consolidado = df_items_consolidado.fillna(0)


df_items_consolidado_sol =df_solicitudes.groupby(['Referencia','Desc. item','Bodega'])['Cant. pendiente'].sum().reset_index()
df_items_consolidado =df_items_consolidado.merge(df_items_consolidado_sol,how='left',on=['Referencia','Desc. item','Bodega'])



df_items_consolidado['Cant. pendiente_y'] = df_items_consolidado.apply(
    lambda row: row['Cant. pendiente_y'] if row['Año'] == 2025 else 0, axis=1)

df_items_consolidado = df_items_consolidado.fillna(0)

df_items_consolidado_mt = df_lista_materiales.groupby(['Referencia','Desc. item','Bodega'])[['Pedidos Acom','Inv.Proyectado']].mean().reset_index()
df_items_consolidado = df_items_consolidado.merge(df_items_consolidado_mt,how='left',on=['Referencia','Desc. item','Bodega'])

# Aplicar la lógica para asignar valores a las columnas de manera adecuada
df_items_consolidado['Pedidos Acom'] = df_items_consolidado.apply(
    lambda row: row['Pedidos Acom'] if row['Año'] == 2025 else 0, axis=1)

df_items_consolidado['Inv.Proyectado'] = df_items_consolidado.apply(
    lambda row: row['Inv.Proyectado'] if row['Año'] == 2025 else 0, axis=1)

# Llenar los valores NaN con 0 si es necesario
df_items_consolidado = df_items_consolidado.fillna(0)

print(df_items_consolidado)

df_items_consolidado = df_items_consolidado.drop_duplicates(
    subset=['Referencia', 'Desc. item', 'CATEGORIA', 'TIPO', 'ORIGEN', 'LINEA', 'Bodega', 'U.M.', 'STATUS', 'Cant. pendiente_x']
)

df_items_consolidado.to_excel('valY.xlsx')

df_items_consolidado.rename(columns={
    'Cant. pendiente_x': 'Cant. pendiente_OCC',
    'Cant. pendiente_y': 'Cant. pendiente_solicitudes',
    'Pedidos Acom': 'Pedidos_Acom',
    'Inv.Proyectado': 'Inv_Proyectado'
}, inplace=True)

'''
df_items_consolidado = df_items_consolidado.groupby(
    ['Referencia', 'Desc. item', 'CATEGORIA', 'TIPO', 'ORIGEN', 'LINEA', 'Bodega', 'U.M.','STATUS'],
    as_index=False  # Evita que las columnas de agrupación se vuelvan índice
).agg({
    'Total_consumo_mean':'mean',
    'Salida_pron_min':'mean',
    'Salida_pron_max':'mean',
    'Salida_pron_mean':'mean',
    'Salida_pron_std':'mean',
    'Salida_pron_median':'mean',
    'Cant. disponible':'mean',
    'Cant. pendiente_x':'sum', 
    'Cant. pendiente_y':'mean',
    'Pedidos Acom':'mean',
    'Inv.Proyectado':'mean',
    'Lead time esperado':'mean'
    }
).rename(columns={'Total_consumo_mean':'Consumos_2024','Salida_pron_min':'Consumo_min','Salida_pron_max':'Consumo_max','Salida_pron_mean':'Consumo_prom','Salida_pron_std':'Desv_consumo','Salida_pron_median':'Consumo_mediana','Cant. pendiente_x':'Cant. pendiente_OCC','Cant. pendiente_y':'Cant. pendiente_solicitudes','Cant. Total': 'Requerimientos_pedidos'})


df_items_consolidado.to_excel('consumoI.xlsx', index=False)
'''
df_items_consolidado.to_excel('consumoI.xlsx', index=False)

  Referencia                                Desc. item             CATEGORIA  \
0    PZAC159             SOPORTE MOTOR PERGOLAM GALAXY  105 - SEMI-ELABORADO   
1    PZAC159             SOPORTE MOTOR PERGOLAM GALAXY  105 - SEMI-ELABORADO   
2    PZCH125    PERFIL EN SILICONA SELLE LAMA PERGOLAM   101 - MATERIA PRIMA   
3      26735        100W OUTPUT SWITCHING FUENTE POWER   101 - MATERIA PRIMA   
4      22537  10393548 GRIS ANTRACITA GRAN MATE X25 KG          102 - INSUMO   

                             TIPO          ORIGEN               LINEA Bodega  \
0              213 - PIEZAS ACERO  801 - COLOMBIA                 NaN  MP001   
1              213 - PIEZAS ACERO  801 - COLOMBIA                 NaN  MP002   
2               201 - COMPONENTES     804 - CHINA  630 - NEW PERGOLAM  MP002   
3  205 - ILUMINACION Y ACCESORIOS  801 - COLOMBIA           699 - N/A  MP002   
4                  229 - PINTURAS  801 - COLOMBIA      618 - PERGOLAM  MP002   

   U.M.     STATUS  Lead time esperado

In [45]:

base_occ_filtrado = df_OCC_HISTORICO[
    df_OCC_HISTORICO['Fecha aprobación solicitud'].notna() &
    df_OCC_HISTORICO['Fecha cumplido'].notna()
].copy()

base_occ_filtrado.to_excel('base_occ_filtrado_sin_fecha.xlsx', index=False)

#base_occ_filtrado['Fecha orden'] = pd.to_datetime(base_occ_filtrado['Fecha orden'], errors='coerce')



base_occ_filtrado["Fecha orden"] = pd.to_datetime(base_occ_filtrado["Fecha orden"], errors="coerce")

base_occ_2024= base_occ_filtrado[base_occ_filtrado["Fecha orden"].dt.year.isin([2024, 2025])]

base_occ_2024['Año']= base_occ_2024['Fecha orden'].dt.year

base_occ_2024.to_excel('base_occ_2024.xlsx')

base_occ_2024["Frecuencia_mes_occ"] = base_occ_filtrado["Fecha orden"].dt.to_period("M")

frecuencia_mes_occ = base_occ_2024.groupby(["Año","Referencia","Bodega"])["Frecuencia_mes_occ"].nunique().reset_index()

frecuencia_ordenes = base_occ_2024[['Fecha orden','Referencia','Bodega','Cant. ordenada','Frecuencia_mes_occ']].copy()



frecuencia_ordenes["Frecuencia_mes_occ"] = frecuencia_ordenes["Frecuencia_mes_occ"].astype(int)




print(type(frecuencia_mes_occ))

frecuencia_mes_occ.to_excel('frecuencia_mes_occ.xlsx')


base_occ_filtrado['Año'] = base_occ_filtrado['Fecha orden'].dt.year

base_occ_filtrado['Lead time'] = base_occ_filtrado['Fecha cumplido'] - base_occ_filtrado['Fecha aprobación solicitud']
base_occ_filtrado['lead time (dias)'] = base_occ_filtrado['Lead time'].dt.days
base_occ_filtrado=base_occ_filtrado[['Año','Referencia','Desc. item','Bodega','lead time (dias)','Cant. ordenada']]
base_occ_filtrado = base_occ_filtrado[(base_occ_filtrado['Bodega'].isin(['MP001','MP002']))]


base_occ_filtrado.to_excel('baseopm.xlsx')


base_occ_filtrado.to_excel('base_occ_filtrado.xlsx', index=False)

base_occ_filtrado['Ordenes'] = (
    base_occ_filtrado.groupby(['Año','Referencia','Bodega'])['Cant. ordenada']
    .transform(lambda x: winsorize_group(x))
)


base_occ_filtrado = base_occ_filtrado.merge(df_items_consolidado[['Año','Referencia', 'Desc. item', 'Bodega','Lead time esperado']], how='left',on=['Año','Referencia','Desc. item','Bodega'])

print(frecuencia_mes_occ.head())


base_occ_filtrado = base_occ_filtrado.merge(frecuencia_mes_occ[['Año','Referencia','Bodega','Frecuencia_mes_occ']], how='left',on=['Año','Referencia','Bodega'])

print(base_occ_filtrado.head())



base_occ_filtrado['lead time winsorized'] = winsorize(base_occ_filtrado['lead time (dias)'], limits=[0.05,0.05])
# 2. Calcular media y desviación estándar de los datos ajustados
mu = base_occ_filtrado['lead time winsorized'].mean()
sigma= base_occ_filtrado['lead time winsorized'].std()

# 3. Normalización: Centrar en 10 días con desviación ajustada a 5 días
sigma_nuevo = 5
sigma_nuevo_sg = 4

base_occ_filtrado['lead time (dias) normalized'] = base_occ_filtrado['Lead time esperado'] + ((base_occ_filtrado['lead time winsorized'] - mu) / sigma) * sigma_nuevo
base_occ_filtrado['lead time (dias) normalized_sig4'] = base_occ_filtrado['Lead time esperado'] + ((base_occ_filtrado['lead time winsorized'] - mu) / sigma) * sigma_nuevo_sg

base_occ_filtrado = base_occ_filtrado.groupby(
    ['Año','Referencia', 'Desc. item','Bodega'],
    as_index=False  # Evita que las columnas de agrupación se vuelvan índice
).agg({
    'Ordenes':['mean','min','max'],
    'Frecuencia_mes_occ':'mean',
    'lead time (dias)':'mean',
    'Lead time esperado':'mean',
    'lead time winsorized':'mean',
    'lead time (dias) normalized':'mean', 
    'lead time (dias) normalized_sig4':['mean','max','median','std'],
    
    
    }
)




base_occ_filtrado.columns = [''.join(col).strip() if isinstance(col, tuple) else col for col in base_occ_filtrado.columns]



base_occ_filtrado.to_excel('base_occ_filtrado.xlsx')

C:\Users\JORGE CONTRERAS\AppData\Local\Temp\ipykernel_32480\3620603689.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_occ_2024['Año']= base_occ_2024['Fecha orden'].dt.year
C:\Users\JORGE CONTRERAS\AppData\Local\Temp\ipykernel_32480\3620603689.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_occ_2024["Frecuencia_mes_occ"] = base_occ_filtrado["Fecha orden"].dt.to_period("M")


<class 'pandas.core.frame.DataFrame'>
    Año Referencia Bodega  Frecuencia_mes_occ
0  2024          3  MP001                   3
1  2024          3  MP002                   3
2  2024          3  MPI01                   1
3  2024          8  MP001                   5
4  2024          8  MP002                   8
    Año Referencia                          Desc. item Bodega  \
0  2021       4948          HILO EPIC 6020-T-120 NEGRO  MP002   
1  2022      13018  RODACHIN RP 130    1   MESA  TUCAN  MP001   
2  2022      13018  RODACHIN RP 130    1   MESA  TUCAN  MP001   
3  2021       8204                          TOPE BALIN  MP001   
4  2021       8204                          TOPE BALIN  MP001   

   lead time (dias)  Cant. ordenada  Ordenes  Lead time esperado  \
0                18             8.0      8.0                 NaN   
1                 4            10.0     10.0                 NaN   
2                43            10.0     10.0                 NaN   
3                19    

In [13]:
df_consumos_año.head(5)

,Año,Referencia,Desc. item,Bodega,Total_consumo,Consumo_maximo,Consumo_minimo,Consumo_promedio,desviacion_estandar
0,2022,3,CARTON SINGLE FACE KILOS,MP001,88.0,88.0,88.0,88.000000,NaN
1,2022,3,CARTON SINGLE FACE KILOS,MP002,7667.0,631.0,1.0,111.115942,111.605863
2,2022,8,CINTA EMPAQUE IMPRESA 48 mm *100 MTS C7,MP002,1248.0,81.0,1.0,11.449541,11.350058
3,2022,9,CINTA EMPAQUE 2 * 100 M TOLDOS TOSCANI,MP002,406.0,19.0,1.0,6.151515,4.476408
4,2022,11,ANGULO HIERRO 1/8 X 1,MP001,4.5,4.5,4.5,4.500000,NaN


In [49]:
dias_operacion=300
df_base_final = df_items_consolidado.merge(base_occ_filtrado, how='left', on=['Año','Referencia','Desc. item','Bodega'])
#df_base_final =df_base_final.drop(columns=['Lead time esperado_y'])

df_base_final.to_excel('df_base_final.xlsx', index=False)

print(df_consumos_año)

df_consumos_año = df_consumos_año[df_consumos_año['Año'] == 2024].reset_index(drop=True)

df_consumos_año.to_excel('consumosO.xlsx')

print(df_consumos_año.columns)


#df_consumos_año = df_consumos_año[df_consumos_año['Año'] == 2024].reset_index(drop=True)




#df_base_final =df_base_final.merge(df_consumos_mes[['Referencia','Desc. item','Bodega','Frecuencia_consumo']],how='left',on=['Referencia','Desc. item','Bodega'])

#df_base_final =df_base_final.merge(df_consumos_año[['Referencia','Desc. item','Bodega','desviacion_estandar']],how='left',on=['Referencia','Desc. item','Bodega'])

#df_base_final =df_base_final.merge(df_total_consumos_mes[['Referencia','Desc. item','Bodega','Consumo_promedio_mes_std']],how='left',on=['Referencia','Desc. item','Bodega'])

print(df_base_final.columns)

df_base_final=df_base_final.drop(columns=['lead time (dias)mean','Lead time esperadomean','lead time winsorizedmean','lead time (dias) normalizedmean'])
df_base_final=df_base_final.fillna(0)
print(df_base_final.columns)
df_base_final['Consumo_diario']=df_base_final['Total_consumo']/dias_operacion
df_base_final['Consumo_diarioT_std'] = df_base_final['Consumo_diario'] /np.sqrt(25)
df_base_final['Consumo_diario_mes']=df_base_final['Total_consumo']/df_base_final['Frecuencia_consumo']
df_base_final['Consumo_diario_diario']=df_base_final['Consumo_diario_mes']/25
df_base_final['Stock_Seguridad'] = 1.65 * np.sqrt(((df_base_final['Consumo_diario_diario']**2) * df_base_final['lead time (dias) normalized_sig4std']**2)+((df_base_final['lead time (dias) normalized_sig4mean']**2)*(df_base_final['Consumo_diarioT_std']**2)))
df_base_final['Punto_reorden'] = (df_base_final['Consumo_diario_diario'])*df_base_final['lead time (dias) normalized_sig4mean'] + df_base_final['Stock_Seguridad']
df_base_final['nivel_minimo_inv'] = df_base_final['Stock_Seguridad']
df_base_final['nivel_maximo_inv'] = df_base_final['Stock_Seguridad'] +((df_base_final['Consumo_diario_diario']*df_base_final['lead time (dias) normalized_sig4mean']))+((df_base_final['Consumo_diario_diario']*(df_base_final['Ordenesmean']/ df_base_final['Consumo_diario_diario'])))
df_base_final['nivel_optimo_solicitar']=(df_base_final['nivel_maximo_inv']+df_base_final['nivel_minimo_inv'])/2
df_base_final['Coonsumo_mensual']=df_base_final['Consumo_diario']*25
df_base_final['Cubrimiento_Dias'] = (df_base_final['Cant. disponible']-df_base_final['Pedidos_Acom'])/df_base_final['Consumo_diario']
df_base_final['Cubrimiento_mes'] = df_base_final['Cubrimiento_Dias'] / 25  # Convertir a meses
df_base_final=df_base_final.fillna(0)



libro = load_workbook(r"c:/Users/JORGE CONTRERAS/OneDrive - 900208659-2 DAMIS SAS/Escritorio/PLANEACION/PLANEACION/INDICADORES PLANEACION/BASES_INDICADORES/Analisis_Max_min.xlsx")

if 'consolidado' in libro.sheetnames:
    del libro['consolidado']

libro.save(r"c:/Users/JORGE CONTRERAS/OneDrive - 900208659-2 DAMIS SAS/Escritorio/PLANEACION/PLANEACION/INDICADORES PLANEACION/BASES_INDICADORES/Analisis_Max_min.xlsx")

with pd.ExcelWriter(r"c:/Users/JORGE CONTRERAS/OneDrive - 900208659-2 DAMIS SAS/Escritorio/PLANEACION/PLANEACION/INDICADORES PLANEACION/BASES_INDICADORES/Analisis_Max_min.xlsx", engine='openpyxl', mode='a') as writer:
    df_base_final.to_excel(writer, sheet_name='consolidado', index=False)





       Año Referencia                                Desc. item Bodega  \
0     2024          3                  CARTON SINGLE FACE KILOS  MP001   
1     2024          3                  CARTON SINGLE FACE KILOS  MP002   
2     2024          8   CINTA EMPAQUE IMPRESA 48 mm *100 MTS C7  MP001   
3     2024          8   CINTA EMPAQUE IMPRESA 48 mm *100 MTS C7  MP002   
4     2024          9   CINTA EMPAQUE  2 * 100 M TOLDOS TOSCANI  MP002   
...    ...        ...                                       ...    ...   
1950  2024    TUBSM03     TUBO RANURADO 70mm 4m (66015) TOSCANY  MP002   
1951  2024    TUBSM04  TUBO CUADRADO 40X40mm 5.85m (66016)TOSCA  MP002   
1952  2024    TUBSM05   TUBO CUADRADO 40X40mm 5m (66016)TOSCANY  MP002   
1953  2024    TUBSM06   TUBO CUADRADO 40X40mm 4m (66016)TOSCANY  MP002   
1954  2024    TUBSM07   TUBO RANURADO 80mm 5.8 Mts (66015)TOSCN  MP002   

      Total_consumo  Consumo_min  Consumo_max  Consumo_promedio  \
0            990.08         41.0         45.

KeyError: 'Frecuencia_consumo'

In [15]:
df_consumos=df_consumos[df_consumos['Referencia']==6543]
df_consumos

,Mes,Orden interno,Referencia,Fecha,Documento,Desc. item,Bodega,Lote,Ubicación,Entradas (inv.),Salidas (inv.),Mes-Año
Año,,,,,,,,,,,,
2023,oct,10,6543,2023-10-13,SCI-00001251,SIKASIL IA TRANSPARENTE,MP002,NaN,NaN,0,2.0,Oct-23
2023,dic,12,6543,2023-12-26,SCI-00050374,SIKASIL IA TRANSPARENTE,MP002,NaN,NaN,0,2.0,Dec-23
2023,oct,10,6543,2023-10-25,SCI-00001281,SIKASIL IA TRANSPARENTE,MP002,NaN,NaN,0,1.0,Oct-23
2023,nov,11,6543,2023-11-03,SCI-00001314,SIKASIL IA TRANSPARENTE,MP002,NaN,NaN,0,1.0,Nov-23
2023,ago,8,6543,2023-08-11,SCI-00000982,SIKASIL IA TRANSPARENTE,MP002,NaN,NaN,0,1.0,Aug-23
...,...,...,...,...,...,...,...,...,...,...,...,...
2024,jul,7,6543,2024-07-02,REM-00049249,SIKASIL IA TRANSPARENTE,MP002,NaN,NaN,0,1.0,Jul-24
2024,oct,10,6543,2024-10-25,REM-00050137,SIKASIL IA TRANSPARENTE,MP002,NaN,NaN,0,1.0,Oct-24
2024,may,5,6543,2024-05-29,REM-00049029,SIKASIL IA TRANSPARENTE,MP002,NaN,NaN,0,3.0,May-24
